### Transport word problem -> LP -> Dual

![original problem](Screenshot_2023-08-17_13-49-14.png)

![details](Screenshot_2023-08-17_13-55-38.png)

Note that the matrices that describe the inequality isn't 3x4. It's 3x12. You need to think of representing every decision variable in the inequality. Same for the equality, which will be 4x12.

Let's say we model the problem and are also given some updated values for the cost matrix.

![updated_details](Screenshot_2023-08-17_14-45-10.png)

In [19]:
### LP

stock = [60,150,190];
demand = [75,120,80,30]; % from the notes: [30,80,120,75]

C = [20,28,14,25,
     10,12,19,8;
     17,11,23,14];
% A simple reshape will not do the job, because matrices are read down rows first, but we need it rows first
C = reshape(C', 12,1);

AI = zeros(3,12);
% i is for depo, j is for store
% supply is the inequality
for i=1:3
    for j=1:4
        AI(i,j+(i-1)*4)=1;
    end
end

AE = zeros(4,12);
% demand is the equality
for i=1:4
    for j=1:3
        AE(i,(j-1)*4+i)=1;
    end
end

lb = zeros(1,12);

A = [AI;AE];
b = [stock, demand];

param.dual = 3; # dual simplex

ctype = "UUUSSSS"; # all upper bounds
vartype = "CCCCCCCCCCCC"; # continuous variable, usually you don't want an integer variable
sense = 1; # -1 is for maximimsation

[x, fmin, status, extra] = glpk(C, A, b, lb, [], ctype, vartype, sense, param);

fmin
extra.lambda
x


fmin = 3530
ans =

   -5
    0
    0
   10
   11
   19
    8

x =

     0
     0
    60
     0
    75
     0
    20
    30
     0
   120
     0
     0



![LP to Dual](Screenshot_2023-08-17_16-28-59.png)

In [30]:
### Dual

Dual_f = [-stock,demand];

Dual_ineq_LHS = [-AI; AE]; #Apply reverse sign to inequalities


lb = [0,0,0,-inf,-inf,-inf,-inf];
ub = [inf,inf,inf,inf,inf,inf,inf];

param.dual = 3; # dual simplex

ctype = "UUUUUUUUUUUU"; # all upper bounds
vartype = "CCCCCCC"; # continuous variable, usually you don't want an integer variable
sense = -1; # -1 is for maximimsation

[Dual_x, Dual_fmin, Dual_status, Dual_extra] = glpk(Dual_f', Dual_ineq_LHS', C', lb, ub, ctype, vartype, sense, param);

Dual_extra.lambda
Dual_x

ans =

     0
     0
    60
     0
    75
     0
    20
    30
     0
   120
     0
     0

Dual_x =

    5
    0
    0
   10
   11
   19
    8



The nature of this is that the Dual and the original program should have the same fmin.

The lambda of the original LP should be the objective values of the Dual, and vice-versa.

In [28]:
assert(fmin == Dual_fmin);
assert(Dual_extra.lambda == x);

# This one assertion fails, why is there one element that's positive and one negative.

extra.lambda == Dual_x

ans =

  0
  1
  1
  1
  1
  1
  1

